```{autolink-concat}
```

::::{margin}
:::{card} Investigation of acceptance matrix with Dalitz-plot decomposition
TR-035
^^^
We investigate how to compute the acceptance matrix for a three-body decay with a spinful final state ($J/\psi \to \bar{p} K^0_S \Sigma^+$). The acceptance matrix then becomes a complex-valued matrix.
:::
::::

# DPD acceptance matrix

<!-- cspell:ignore bmatrix lognormal rcccc -->

In [ ]:
import itertools
import logging
import re
import warnings
from collections.abc import Iterable
from itertools import product

import attrs
import graphviz
import jax.numpy as jnp
import numpy as np
import qrules
import sympy as sp
from ampform.dynamics.form_factor import FormFactor
from ampform.kinematics.lorentz import (
    Energy,
    EuclideanNorm,
    FourMomentumSymbol,
    InvariantMass,
    ThreeMomentum,
)
from ampform.sympy import PoolSum
from ampform.sympy._decorator import unevaluated
from ampform_dpd import AmplitudeModel, DalitzPlotDecompositionBuilder
from ampform_dpd.adapter.qrules import normalize_state_ids, to_three_body_decay
from ampform_dpd.decay import IsobarNode, Particle, State, ThreeBodyDecayChain
from ampform_dpd.dynamics import RelativisticBreitWigner
from ampform_dpd.io import cached, simplify_latex_rendering
from IPython.display import HTML, display
from tensorwaves.data.phasespace import TFPhaseSpaceGenerator
from tensorwaves.data.rng import TFUniformRealNumberGenerator
from tensorwaves.data.transform import SympyDataTransformer
from tensorwaves.interface import DataSample, ParametrizedFunction

logging.getLogger().setLevel(logging.ERROR)
np.set_printoptions(linewidth=120)
simplify_latex_rendering()
warnings.simplefilter("ignore", category=RuntimeWarning)

In [ ]:
display(HTML("<style>.container { width:100% !important; }</style>"))

## Introduction


To speed up the fit the part of the negative log likelihood (NLL) function representing the normalization computed over the  Monte-Carlo phsp can be pre-computed. Note that this method can only be applied in the _specific case that masses and widths are fixed_ as the normalization depends on these parameters.

Statistically this approach can be derived as follows:
Given total sample $\vec{x}$ consisting of _N_ independent observations of a set the $n$ quantities $x$ depending the likelihood function can be written as the product of the (Probability Density Functions) PDFs $f$ of each single observation:

$$L(\vec{x} ; \vec{\theta})=\prod_{\mathfrak{i}=1}^N f\left(x_i ; \vec{\theta}\right)$$

$\vec{\theta}$ represents the set of $m$ unknown parameters the chosen PDF depends on.
In the case of the this amplitude analysis the PDFs are the normalized intensity functions for each partial wave:

$$f(x_i ; \vec{\theta})= \frac{I(x_i ; \vec{\theta})}{\int I(\vec{x} ; \vec{\theta}) d\vec{x}}$$ 

 The $x_i$ represent the selected events containing the information about the Mandelstam variables and the decay and production angles. With the expression $f$ the NLL function is given as:

$$
NLL(x_i ; \vec{\theta})=-\sum^n_{i=1} \log \left(\frac{I(x_i ; \vec{\theta})}{\int I(\vec{x} ; \vec{\theta}) d\vec{x}}\right) = \underbrace{-\sum^n_{i=1} \log(I(x_i ; \vec{\theta}))}_{\mathrm{Data}} + n\cdot \log\left(\int I(\vec{x} ; \vec{\theta}) d\vec{x} \right)
$$

Since the second term integrates over all phase-space points, it is independent of \( x_i \) and can be approximated using Monte Carlo integration:

$$
\int I(\vec{x} \,; \vec{\theta}) \, d\vec{x} = 
c^{\dagger} \cdot \frac{1}{N_{\mathrm{MC}}} \sum_{j=1}^{N_{\mathrm{MC}}} I(\vec{x}_j^{\mathrm{MC}} \,; \vec{\theta}) \cdot c 
\cdot \left( \int d\vec{x} \right)
$$

The term $\frac{1}{N_{\mathrm{MC}}} \sum_{j=1}^{N_{\mathrm{MC}}} I(\vec{x}_j^{\mathrm{MC}} ; \vec{\theta})$ represents the so-called acceptance matrix, a hermitian matrix which we call $X$ from now on. The acceptance matrix can be used obtain integrated intensity via the **bilinear relation**:

$$
\int I(\vec{x}) \, \mathrm{d}\vec{x}=c^{\dagger} X c \,.
$$

### Construction of the acceptance matrix

The acceptance matrix can be constructed by probing with four different value combinations for the couplings $c_i,c_j$, which leads to the four equations:

$$
\begin{array}{rcccc}
    A_{ij}^{+} &=&
    \int I(c_i=1, c_j=1) \, \mathrm{d}\vec{x} &=&
    \begin{bmatrix} 1 & 1 \end{bmatrix}
    \begin{bmatrix}
        X_{ii} & X_{ij} \\
        X_{ji} & X_{jj} \\
    \end{bmatrix}
    \begin{bmatrix} 1 \\ 1 \end{bmatrix} \\
    A_{ij}^{-} &=&
    \int I(c_i=1, c_j=-1) \, \mathrm{d}\vec{x} &=&
    \begin{bmatrix} 1 & -1 \end{bmatrix}
    \begin{bmatrix}
        X_{ii} & X_{ij} \\
        X_{ji}& X_{jj} \\
    \end{bmatrix}
    \begin{bmatrix} 1 \\ -1 \end{bmatrix} \\
    B_{ij}^{+} &=&
    \int I(c_i=1, c_j=i) \, \mathrm{d}\vec{x} &=&
    \begin{bmatrix} 1 & i \end{bmatrix}
    \begin{bmatrix}
        X_{ii} & X_{ij} \\
        X_{ji}& X_{jj} \\
    \end{bmatrix}
    \begin{bmatrix} 1 \\ i \end{bmatrix} \\
    B_{ij}^{-} &=&
    \int I(c_i=1, c_j=-i) \, \mathrm{d}\vec{x} &=&
    \begin{bmatrix} 1 & -i \end{bmatrix}
    \begin{bmatrix}
        X_{ii} & X_{ij} \\
        X_{ji}& X_{jj} \\
    \end{bmatrix}
    \begin{bmatrix} 1 \\ -i \end{bmatrix}
\,.
\end{array}
$$

The values $A_{ij}$ and $B_{ij}$ are the elements of 'sub-intensity' matrices,, which can be computed by setting $c_i$ and $c_j$ to the respective values, setting the other couplings to zero, and integrating over the phase space. Finally, the elements of $X$ can be expressed as:

$$
X_{ij} = \frac{A_{ij}^+-A_{ij}^-}{4} + i\cdot\frac{B_{ij}^+-B_{ij}^-}{4} \,.
$$

As you can see in the expression above, two equations are needed for the real part of $X_{ij}$ and two for the imaginary part.

### Speed up the construction of $X$

Using the four equations described above is not computationally efficient, we have to compute the sub-intensity matrix for all $4m^2$ combinations of $c_i$ and $c_j$, with $m$ the number of couplings. However, we can use the hermitian property of the acceptance matrix to reduce the required number of equations to construct the matrix. A hermitian matrix is mirror symmetric with respect to its main diagonal, up to the complex conjugation of all entries. Therefore one only has to calculate the upper triangle of $X$ and then fill the lower triangle with its complex-conjugate transpose. As probes the vectors with $c_i=1,c_j=1$ are used to obtain the real part, while $c_i=i,c_j=1$ is used to obtain the imaginary part of the elements of $X_{ij}$. The diagonal of $X$ is real-valued and is equal to the branching fractions (that is, the diagonal of the sub-intensity matrix with $c_i=c_j=1$).

With this, we can compute the acceptance matrix as

$$
X_{ij} = \frac{A_{c_i=1,c_j=1}+A_{c_i=i,c_j=1}-(1 + i) (\text{diagonal}[i] + \text{diagonal}[j])}{2} \,.
$$

## Prepare model and data

### Define reaction

In [ ]:
PDG = qrules.load_default_particles()
PDG.add(
    qrules.particle.create_particle(
        template_particle=PDG.find("N(1720)+"),
        name="N(2060)+",
        mass=2.1,
        width=0.4,
        pid=200004,
        parity=-1,
        spin=5 / 2,
        latex="N(2060)^+",
    ),
)

In [ ]:
reaction = qrules.generate_transitions(
    initial_state=[("J/psi(1S)", [-1, +1])],
    final_state=["K0", "Sigma+", "p~"],
    allowed_interaction_types="strong",
    allowed_intermediate_particles=[
        "N(2060)",
        "Sigma(1750)",
        "Sigma(1775)",
    ],
    mass_conservation_factor=0,
    particle_db=PDG,
)
reaction = normalize_state_ids(reaction)
dot = qrules.io.asdot(reaction, collapse_graphs=True)
graphviz.Source(dot)

### Define amplitude model

In [ ]:
def formulate_breit_wigner_with_ff(
    chain: ThreeBodyDecayChain,
) -> tuple[sp.Expr, dict[sp.Symbol, float]]:
    s = _get_mandelstam_s(chain)
    parameter_defaults = {}
    production_ff, new_pars = _create_form_factor(s, chain.production_node)
    parameter_defaults.update(new_pars)
    decay_ff, new_pars = _create_form_factor(s, chain.decay_node)
    parameter_defaults.update(new_pars)
    breit_wigner, new_pars = _create_breit_wigner(s, chain.decay_node)
    parameter_defaults.update(new_pars)
    return (
        production_ff * decay_ff * breit_wigner,
        parameter_defaults,
    )


def _create_form_factor(
    s: sp.Symbol,
    isobar: IsobarNode,
) -> tuple[sp.Expr, dict[sp.Symbol, float]]:
    assert isobar.interaction is not None, "Need LS-couplings"
    inv_mass = _generate_mass_symbol(isobar.parent, s)
    outgoing_state_mass1 = _generate_mass_symbol(_get_particle(isobar.child1), s)
    outgoing_state_mass2 = _generate_mass_symbol(_get_particle(isobar.child2), s)
    meson_radius = _create_meson_radius_symbol(isobar.parent)
    form_factor = FormFactor(
        s=inv_mass**2,
        m1=outgoing_state_mass1,
        m2=outgoing_state_mass2,
        angular_momentum=isobar.interaction.L,
        meson_radius=meson_radius,
    )
    parameter_defaults = {
        meson_radius: 1,
    }
    return form_factor, parameter_defaults


def _generate_mass_symbol(state: State | Particle, s: sp.Symbol) -> sp.Symbol:
    if isinstance(state, State):
        return create_mass_symbol(state)
    return sp.sqrt(s)


def _create_breit_wigner(
    s: sp.Symbol,
    isobar: IsobarNode,
) -> tuple[sp.Expr, dict[sp.Symbol, float]]:
    assert isobar.interaction is not None, "Need LS-couplings"
    outgoing_state_mass1 = create_mass_symbol(isobar.child1)
    outgoing_state_mass2 = create_mass_symbol(isobar.child2)
    angular_momentum = isobar.interaction.L
    res_mass = create_mass_symbol(isobar.parent)
    res_width = sp.Symbol(Rf"\Gamma_{{{isobar.parent.latex}}}", nonnegative=True)
    meson_radius = _create_meson_radius_symbol(isobar.parent)

    breit_wigner_expr = RelativisticBreitWigner(
        s=s,
        mass0=res_mass,
        gamma0=res_width,
        m1=outgoing_state_mass1,
        m2=outgoing_state_mass2,
        angular_momentum=angular_momentum,
        meson_radius=meson_radius,
    )
    parameter_defaults = {
        res_mass: isobar.parent.mass,
        res_width: isobar.parent.width,
        meson_radius: 1,
    }
    return breit_wigner_expr, parameter_defaults


def _create_meson_radius_symbol(isobar: IsobarNode) -> sp.Symbol:
    particle = _get_particle(isobar)
    if isinstance(particle, State):
        if particle.index != 0:
            msg = "Only the initial state has a meson radius"
            raise NotImplementedError(msg)
        return sp.Symbol(R"R_{J/\psi}")
    return sp.Symbol(Rf"R_\mathrm{{{particle.latex}}}")


def create_mass_symbol(particle: IsobarNode | Particle) -> sp.Symbol:
    particle = _get_particle(particle)
    if isinstance(particle, State):
        return sp.Symbol(f"m{particle.index}", nonnegative=True)
    return sp.Symbol(f"m_{{{particle.latex}}}", nonnegative=True)


def _get_mandelstam_s(decay: ThreeBodyDecayChain) -> sp.Symbol:
    s1, s2, s3 = sp.symbols("sigma1:4", nonnegative=True)
    m1, m2, m3 = map(create_mass_symbol, decay.final_state)
    decay_masses = {create_mass_symbol(p) for p in decay.decay_products}
    if decay_masses == {m2, m3}:
        return s1
    if decay_masses == {m1, m3}:
        return s2
    if decay_masses == {m1, m2}:
        return s3
    msg = f"Cannot find Mandelstam variable for {''.join(decay_masses)}"
    raise NotImplementedError(msg)


def _get_particle(isobar: IsobarNode | State) -> State | Particle:
    if isinstance(isobar, IsobarNode):
        return isobar.parent
    return isobar

In [ ]:
def prepare_for_phsp(model: AmplitudeModel) -> AmplitudeModel:
    p1, p2, p3 = (FourMomentumSymbol(f"p{i}", shape=[]) for i in (1, 2, 3))
    s1, s2, s3 = sp.symbols("sigma1:4", nonnegative=True)
    mass_definitions = {
        s1: InvariantMassSquared(p2 + p3),
        s2: InvariantMassSquared(p1 + p3),
        s3: InvariantMassSquared(p1 + p2),
        sp.Symbol("m0", nonnegative=True): InvariantMass(p1 + p2 + p3),
        sp.Symbol("m1", nonnegative=True): InvariantMass(p1),
        sp.Symbol("m2", nonnegative=True): InvariantMass(p2),
        sp.Symbol("m3", nonnegative=True): InvariantMass(p3),
    }
    mass_definitions = {k: sp.sympify(v) for k, v in mass_definitions.items()}
    angle_and_mandelstam_definitions = {
        symbol: expr.xreplace(mass_definitions)
        for symbol, expr in model.variables.items()
    }
    angle_and_mandelstam_definitions.update(mass_definitions)
    polarized_intensity = set_initial_state_polarization(
        model.intensity,
        spin_projections=(-1, +1),
    )
    new_parameter_defaults = {
        symbol: v
        for symbol, v in model.parameter_defaults.items()
        if not re.match(r"m[0-3]", symbol.name)
    }
    return attrs.evolve(
        model,
        intensity=polarized_intensity,
        variables=angle_and_mandelstam_definitions,
        parameter_defaults=new_parameter_defaults,
    )


def set_initial_state_polarization(
    intensity: PoolSum, spin_projections: Iterable[sp.Rational | float]
) -> PoolSum:
    helicity_symbol, _ = intensity.indices[0]
    helicity_values = tuple(sp.Rational(i) for i in spin_projections)
    new_indices = (
        (helicity_symbol, helicity_values),
        *intensity.indices[1:],
    )
    return PoolSum(intensity.expression, *new_indices)


@unevaluated
class InvariantMassSquared(sp.Expr):
    momentum: sp.Basic
    _latex_repr_ = R"M^2\left({momentum}\right)"

    def evaluate(self) -> sp.Expr:
        p = self.momentum
        p_xyz = ThreeMomentum(p)
        return Energy(p) ** 2 - EuclideanNorm(p_xyz) ** 2

In [ ]:
decay = to_three_body_decay(reaction.transitions, min_ls=False)
builder = DalitzPlotDecompositionBuilder(decay, min_ls=False)
for chain in builder.decay.chains:
    builder.dynamics_choices.register_builder(chain, formulate_breit_wigner_with_ff)
model = builder.formulate(reference_subsystem=2, cleanup_summations=True)
model = prepare_for_phsp(model)
model.intensity.cleanup()

### Generate phase space sample

In [ ]:
dpd_transformer = SympyDataTransformer.from_sympy(model.variables, backend="jax")

In [ ]:
phsp_generator = TFPhaseSpaceGenerator(
    initial_state_mass=reaction.initial_state[0].mass,
    final_state_masses={i: p.mass for i, p in reaction.final_state.items()},
)
rng = TFUniformRealNumberGenerator(seed=0)
phsp = phsp_generator.generate(100_000, rng)
phsp = dpd_transformer(phsp)
phsp = {k: phsp[k] for k in sorted(phsp)}
phsp = {k: v if jnp.iscomplex(v).any() else v.real for k, v in phsp.items()}

In [ ]:
phsp

### Prepare numerical function

In [ ]:
full_intensity_expr = cached.unfold(model)

In [ ]:
fixed_parameters = {
    symbol: value
    for symbol, value in model.parameter_defaults.items()
    if "production" not in str(symbol)
}
full_intensity_expr = cached.xreplace(full_intensity_expr, fixed_parameters)

In [ ]:
coupling_parameters = {
    symbol: value
    for symbol, value in model.parameter_defaults.items()
    if symbol not in fixed_parameters
}

In [ ]:
intensity_func = cached.lambdify(
    full_intensity_expr,
    parameters=coupling_parameters,
    backend="jax",
)

## Compute acceptance matrix

### Brute-force computation

In [ ]:
def compute_intensity_matrix(
    func: ParametrizedFunction, phsp: DataSample, ci: complex, cj: complex
) -> np.ndarray:
    original_parameters = func.parameters.copy()
    coupling_parameters = [p for p in func.parameters if "production" in p]
    n = len(coupling_parameters)
    sub_intensity_matrix = np.zeros((n, n))
    for (i, c1), (j, c2) in itertools.product(
        enumerate(coupling_parameters),
        enumerate(coupling_parameters),
    ):
        new_parameters = dict.fromkeys(coupling_parameters, 0)
        if c1 == c2:
            new_parameters[c1] = 1
        else:
            new_parameters[c1] = ci
            new_parameters[c2] = cj
        func.update_parameters(new_parameters)
        sub_intensities = func(phsp)
        func.update_parameters(original_parameters)
        sub_intensity_matrix[i, j] = integrate_intensity(sub_intensities)
    return sub_intensity_matrix

In [ ]:
def integrate_intensity(intensities: jnp.ndarray) -> float:
    flattened_intensities = intensities.flatten()
    non_nan_intensities = flattened_intensities[~jnp.isnan(flattened_intensities)]
    return float(jnp.sum(non_nan_intensities) / len(non_nan_intensities))

In [ ]:
%%time
A_plus = compute_intensity_matrix(intensity_func, phsp, ci=1, cj=+1)
B_plus = compute_intensity_matrix(intensity_func, phsp, ci=1, cj=+1j)
A_minus = compute_intensity_matrix(intensity_func, phsp, ci=1, cj=-1)
B_minus = compute_intensity_matrix(intensity_func, phsp, ci=1, cj=-1j)

In [ ]:
X_temp = (A_plus - A_minus) / 4 + 1j / 4 * (B_plus - B_minus)
diagonal = np.tril(A_plus) - np.tril(A_plus, k=-1)
X_brute_force = diagonal + np.tril(X_temp, k=-1) + np.triu(X_temp, k=1)
X_brute_force.round(7)

In [ ]:
np.testing.assert_almost_equal(X_brute_force, X_brute_force.T.conj())

### Smarter computation

In [ ]:
def compute_acceptance_matrix(
    intensity_func: ParametrizedFunction,
    phsp: DataSample,
) -> np.ndarray:
    a_matrix = compute_intensity_matrix_upper_triangle(intensity_func, phsp, ci=1, cj=1)
    b_matrix = compute_intensity_matrix_upper_triangle(
        intensity_func, phsp, ci=1j, cj=1
    )
    diagonal = compute_intensity_matrix_diagonal(intensity_func, phsp)
    x_matrix = np.zeros(a_matrix.shape, dtype=complex)
    for i, j in itertools.product(range(a_matrix.shape[0]), range(a_matrix.shape[1])):
        if i < j:
            x_matrix[j, i] = (
                a_matrix[i, j]
                + 1j * b_matrix[i, j]
                - (1 + 1j) * (diagonal[i] + diagonal[j])
            ) / 2
    conjugate_upper_triangle = x_matrix.conjugate().T
    x_matrix += np.diag(diagonal) + conjugate_upper_triangle
    return x_matrix


def compute_intensity_matrix_upper_triangle(
    func: ParametrizedFunction, phsp: DataSample, ci: complex, cj: complex
) -> np.ndarray:
    original_parameters = func.parameters.copy()
    coupling_parameters = [p for p in func.parameters if "production" in p]
    n = len(coupling_parameters)
    sub_intensity_matrix = np.zeros((n, n))
    for (i, c1), (j, c2) in itertools.product(
        enumerate(coupling_parameters),
        enumerate(coupling_parameters),
    ):
        if i >= j:
            continue
        new_parameters = dict.fromkeys(coupling_parameters, 0)
        new_parameters[c1] = ci
        new_parameters[c2] = cj

        func.update_parameters(new_parameters)
        sub_intensities = func(phsp)
        func.update_parameters(original_parameters)

        sub_intensity_matrix[i, j] = integrate_intensity(sub_intensities)

    return sub_intensity_matrix


def compute_intensity_matrix_diagonal(
    func: ParametrizedFunction, phsp: DataSample
) -> np.ndarray:
    original_parameters = func.parameters.copy()
    coupling_parameters = [p for p in func.parameters if "production" in p]
    n = len(coupling_parameters)
    sub_intensity_array = np.zeros(n)

    for i, c1 in enumerate(coupling_parameters):
        for j in range(i, n):
            c2 = coupling_parameters[j]
            new_parameters = dict.fromkeys(coupling_parameters, 0)
            if c1 == c2:
                new_parameters[c1] = 1
            else:
                continue

            func.update_parameters(new_parameters)
            sub_intensities = func(phsp)
            func.update_parameters(original_parameters)

            sub_intensity_array[i] = integrate_intensity(sub_intensities)

    return sub_intensity_array

In [ ]:
%%time
X = compute_acceptance_matrix(intensity_func, phsp)

In [ ]:
X.round(7)

In [ ]:
np.testing.assert_almost_equal(X, X.T.conj())
np.testing.assert_almost_equal(X, X_brute_force)

## Test bilinear form 

The the bilinear form with the acceptance matrix is to be used within for the optimization of the coupling parameters $c_{ij}$. The bilinear form should lead to the same result when calculating the sub-intensity directly by Monte-Carlo integration. In the following this expected behavior is tested for different $c$.

Result taking every element of $c$ as $2+1i$ or 0 \
**Direct integration:**

In [ ]:
parameter_names = list(intensity_func.parameters)
n = len(parameter_names)
all_c = [np.array(c, dtype=complex) for c in product([0, -2 + 1j], repeat=n) if any(c)]

integration_results = []
for c in all_c:
    new_parameters = dict(zip(parameter_names, c, strict=True))
    intensity_func.update_parameters(new_parameters)
    intensity_value = integrate_intensity(intensity_func(phsp))
    integration_results.append((tuple(c.astype(complex)), intensity_value))

for c_tuple, value in integration_results:
    print(f"c = {c_tuple} : {value}")

**Bilinear form:**

In [ ]:
bilinear_results = []
for c in all_c:
    c_dagger = c.conj().T
    result = c_dagger @ X @ c
    bilinear_results.append((tuple(c.astype(complex)), result))

for c_tuple, value in bilinear_results:
    print(f"c = {c_tuple} : {value}")

**Difference:**

In [ ]:
for (c, bilinear), (_, integration) in zip(
    bilinear_results, integration_results, strict=True
):
    print(f"{c} : {bilinear == integration!r:6s} {bilinear - integration}")

Now new values for the elements of $c$ are generated by varying the magnitude of the $c_i$ around one according the a log-normal distribution and the complex phase according to a uniform distribution. **Note that this time each $c_i$ has a different phase and magnitude.**

In [ ]:
n_tests = 2
rng = np.random.default_rng(seed=0)
for _ in range(n_tests):
    c_abs = rng.lognormal(mean=1, sigma=0.1, size=n)
    c_phase = rng.uniform(-np.pi, +np.pi, size=n)
    c = c_abs * np.exp(1j * c_phase)

    new_parameters = dict(zip(parameter_names, c, strict=True))
    intensity_func.update_parameters(new_parameters)

    intensity_value_original = intensity_func(phsp)

    c_dagger = c.conj().T
    intensity_value_bilinear = c_dagger @ X @ c

    difference = intensity_value_bilinear - intensity_value_original
    display(f"Coupling vector:{c}")
    display(f"Difference of bilinear form and integration: {difference.round(40)}")

## Further tests

In [ ]:
parameter_names = list(intensity_func.parameters)
n = len(parameter_names)
all_c = [
    np.array([1, 1, 1, 1], dtype=complex),
    np.array([2, 1, 3, 1], dtype=complex),
    np.array([1 + 1j, 2 + 1j, 3 + 1j, 4 + 1j], dtype=complex),
    np.array([1 + 2j, 2 - 1j, 3 + 0.5j, 4 + 5j], dtype=complex),
]

integration_results = []
for c in all_c:
    new_parameters = dict(zip(parameter_names, c, strict=True))
    intensity_func.update_parameters(new_parameters)
    intensity_value = integrate_intensity(intensity_func(phsp))
    integration_results.append((tuple(c.astype(complex)), intensity_value))

for c_tuple, value in integration_results:
    print(f"c = {c_tuple} : {value}")

In [ ]:
bilinear_results = []
for c in all_c:
    c_dagger = c.conj().T
    result = c_dagger @ X @ c
    bilinear_results.append((tuple(c.astype(complex)), result))

for c_tuple, value in bilinear_results:
    print(f"c = {c_tuple} : {value}")